In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch


torch.manual_seed(10)


df1 =  pd.read_csv('/Users/titouan/Desktop/HDA/Term_2/2-ML/Project/Dataset/metadata_compiled.csv', index_col=None)

#Removing NA cough_detected
df1=df1.loc[df1['status'].notna(),]
df1=df1.loc[df1['cough_detected'].notna(),]
df1= df1.loc[df1['cough_detected'] > 0.8,]
df1=df1.drop(columns=["Unnamed: 0"])
len(df1)


Convert files from webm/ogm to wav format

In [ ]:
from Scripts.convert_files import convert_files_updated

path = "/coughvid_20211012/"

convert_files_updated(path, df1)

print("Webm to wav DONE")


Denoising of the audio files

In [ ]:
import os
import noisereduce as nr
from scipy.io import wavfile
from Scripts.convert_files import is_valid_wav  # Make sure this returns True only for valid PCM .wav files

# Paths
path = "/Users/titouan/Desktop/HDA/Term_2/2-ML/Project/coughvid_20211012/"
path_denoised = "/Users/titouan/Desktop/HDA/Term_2/2-ML/Project/denoised_data/"

# Loop over UUIDs
for fn in df1['uuid']:
    wav_file = os.path.join(path, fn + ".wav")
    out_file = os.path.join(path_denoised, fn + ".wav")

    # Skip if already processed
    if os.path.exists(out_file):
        continue

    # Skip if not a valid .wav file
    if not is_valid_wav(wav_file):
        print(f"Invalid .wav file skipped: {fn}")
        continue

    print(f"Processing: {fn}")
    try:
        rate, data = wavfile.read(wav_file)

        if len(data.shape) > 1:
            nframes, nchannels = data.shape
            reduced_data = nr.reduce_noise(y=data.T, sr=rate)
            wavfile.write(out_file, rate, reduced_data.T)
        else:
            reduced_data = nr.reduce_noise(y=data, sr=rate)
            wavfile.write(out_file, rate, reduced_data)

    except Exception as e:
        print(f"Error processing {fn}: {e}")

# Remove macOS .DS_Store file if it exists in output folder
ds_store = os.path.join(path_denoised, ".DS_Store")
if os.path.exists(ds_store):
    os.remove(ds_store)


In [22]:
uuid_ok= [fn[:-4] for fn in os.listdir(path_denoised)]
len(uuid_ok)
# Only keep rows where uuid is in the list of successful denoised files
df1 = df1[df1['uuid'].isin(uuid_ok)]

Remove silent part and create split of distinct coughs

In [ ]:
from unsilence import Unsilence
import pandas as pd
import librosa
from scipy.io import wavfile
from tqdm import tqdm

path_denoised = "/Users/titouan/Desktop/HDA/Term_2/2-ML/Project/denoised_data/"
separated_audio = "/Users/titouan/Desktop/HDA/Term_2/2-ML/Project/Separated_audio/"

label = pd.DataFrame(columns=["uuid","label","non_silent_name"])
path_label = "/Users/titouan/Desktop/HDA/Term_2/2-ML/Project/label_cut_audio.csv"

counter=0
for fn in tqdm(os.listdir(path_denoised)):
    u = Unsilence(path_denoised + fn)
    u.detect_silence(short_interval_threshold=0.5)
    audio, sample_rate = librosa.load(path_denoised + fn)
    for instant in u.get_intervals().serialize():
        #Only keep non-silent part
        if not instant['is_silent']:
            #Crop from start to end
            cropped_audio = audio[int(instant['start']*sample_rate//1-1) :int(instant['end']*sample_rate//1 +1)  ]
            wavfile.write(separated_audio + str(counter) + ".wav", sample_rate, cropped_audio)
            status = df1[df1["uuid"]==fn[:-4]]['status'].values[0]
            label.loc[len(label)] = [fn[:-4], status, str(counter) + ".wav"]
            counter+=1

label.to_csv(path_label)
            

100%|██████████| 12745/12745 [56:00<00:00,  3.79it/s] 


Calculate SNR to filter the unusual sound (not clear cough, voices, clipped signals...)

In [ ]:
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import librosa

path_denoised = "/Users/titouan/Desktop/HDA/Term_2/2-ML/Project/denoised_data/"
separated_audio = "/Users/titouan/Desktop/HDA/Term_2/2-ML/Project/Separated_audio/"

path_label = "/Users/titouan/Desktop/HDA/Term_2/2-ML/Project/label_cut_audio.csv"

df=pd.read_csv(path_label)

df['snr'] = 0

def estimate_snr_split(audio, sr, top_db=20):
    """
    Estimate SNR by separating silent and non-silent regions of a short audio segment.
    `top_db` controls sensitivity of silence detection.
    """
    # Identify non-silent intervals
    intervals = librosa.effects.split(audio, top_db=top_db)
    
    if len(intervals) == 0:
        return float('-inf')  # Entire audio is silent

    signal_mask = np.zeros_like(audio, dtype=bool)
    for start, end in intervals:
        signal_mask[start:end] = True

    signal_energy = np.mean(audio[signal_mask] ** 2)
    noise_energy = np.mean(audio[~signal_mask] ** 2)

    if noise_energy == 0:
        return float('inf')  # No noise detected

    snr_db = 10 * np.log10(signal_energy / noise_energy)
    return snr_db


snrs = []
for fn in tqdm(os.listdir(separated_audio)):
    if fn ==".DS_Store":
        continue
    audio, sr = librosa.load(os.path.join(separated_audio, fn), sr=None)
    #Removing empty file -> Need to change labels file as well !
     
    if len(audio)==0:
        os.remove(separated_audio + fn)
        continue

    snr= estimate_snr_split(audio, sr)

    if snr ==np.inf or snr == float('inf'):
        os.remove(separated_audio + fn)
    else: 
        snrs.append(snr)
        df.loc[df['non_silent_name']==fn,'snr']=snr


plt.hist(snrs, bins=50)
plt.xlabel("Estimated SNR (dB)")
plt.ylabel("Count")
plt.title("SNR Distribution of Segments")
plt.show()

In [34]:
#TO RUN to remove the files from df 

#Remove extrema SNR
df_filtered = df[(df['snr'] > 19) & (df['snr'] < 100)]

new_path_label = "/Users/titouan/Desktop/HDA/Term_2/2-ML/Project/label_cut_audio_new.csv"
df_filtered.to_csv(new_path_label)

In [ ]:
new_path_label = "/Users/titouan/Desktop/HDA/Term_2/2-ML/Project/label_cut_audio_new.csv"
df_filtered=pd.read_csv(new_path_label)
separated_audio = "/Users/titouan/Desktop/HDA/Term_2/2-ML/Project/Separated_audio/"

#Remove files filtered by SNR
file_to_remove = [fn for fn in os.listdir(separated_audio) if fn not in df_filtered['non_silent_name'].values]
for fn in file_to_remove :
     os.remove(separated_audio+fn)


Create Train/Test split and creation of melspectrograms

In [ ]:
from sklearn.model_selection import train_test_split

# Split by UUIDs and labels
df_train, df_test = train_test_split(df, test_size=0.2, shuffle=True, stratify=df['label'])

df_train.to_csv("/Users/titouan/Desktop/HDA/Term_2/2-ML/Project/train_cut_label.csv")

df_test.to_csv("/Users/titouan/Desktop/HDA/Term_2/2-ML/Project/test_cut_label.csv")

In [ ]:
#Creation of MelSpec on the Test set (non-augmented)
from Scripts.spec_augment import MelSpecto

separated_audio = "/Users/titouan/Desktop/HDA/Term_2/2-ML/Project/Separated_audio/"

labels_cut_audio_path = "/Users/titouan/Desktop/HDA/Term_2/2-ML/Project/test_cut_label.csv"
mels_path = "/Users/titouan/Desktop/HDA/Term_2/2-ML/Project/MelSpecs_test/"
melspec_labels_path = "/Users/titouan/Desktop/HDA/Term_2/2-ML/Project/MelSpecs_labels_test.csv"

mean_signal_length = 28835

MelSpecto(waves_path = separated_audio,meanSignalLength=mean_signal_length + 1 ,labels_cut_audio_path= labels_cut_audio_path, mels_path =mels_path , melspec_labels_path = melspec_labels_path)



In [ ]:
#Creation of Augmented MelSpec on the Train set (augmented)
from Scripts.pitch_shift import pitchShift

metaDataPath = "/Users/titouan/Desktop/HDA/Term_2/2-ML/Project/train_cut_label.csv"
audioDataPath = "/Users/titouan/Desktop/HDA/Term_2/2-ML/Project/Separated_audio/"
augmentedSignals = "/Users/titouan/Desktop/HDA/Term_2/2-ML/Project/Pitch_Shift_train/"

pitchShift(metaDataPath,
           audioDataPath,
           augmentedSignals)

12808.wav   100 / 4128
6984.wav   200 / 4128
813.wav   300 / 4128
607.wav   400 / 4128
20742.wav   500 / 4128
15649.wav   600 / 4128
20427.wav   700 / 4128
20368.wav   800 / 4128
20589.wav   900 / 4128
12596.wav   1000 / 4128
11958.wav   1100 / 4128
14233.wav   1200 / 4128
3740.wav   1300 / 4128
11999.wav   1400 / 4128
20877.wav   1500 / 4128
13864.wav   1600 / 4128
12979.wav   1700 / 4128
4219.wav   1800 / 4128
1572.wav   1900 / 4128
18074.wav   2000 / 4128
16268.wav   2100 / 4128
2894.wav   2200 / 4128
647.wav   2300 / 4128
9394.wav   2400 / 4128
13832.wav   2500 / 4128
9717.wav   2600 / 4128
16701.wav   2700 / 4128
7538.wav   2800 / 4128
17695.wav   2900 / 4128
20644.wav   3000 / 4128
5286.wav   3100 / 4128
8760.wav   3200 / 4128
3046.wav   3300 / 4128
4678.wav   3400 / 4128
6272.wav   3500 / 4128
10163.wav   3600 / 4128
21009.wav   3700 / 4128
11413.wav   3800 / 4128
9993.wav   3900 / 4128
4232.wav   4000 / 4128
18480.wav   4100 / 4128


In [ ]:
#Creation of Augmented MelSpec on the Train set (augmented)

from Scripts.spec_augment import SpectAugment

#Mean Signal Length
# len_signals=list()
# 
# for f in os.listdir(wavs_signal_augmented):
#     if f ==".DS_Store":
#         continue
#     signal, sr = librosa.load(wavs_signal_augmented + f)
#     len_signals.append(len(signal))
# 
# mean_signal_length = round(np.mean(len_signals))
# print(mean_signal_length)
# print(mean_signal_length/sr)
# # Manually define mean_signal_length
mean_signal_length = 28835

# Filepath for augmented audio (original and pitch shifted audio): input
wavs_signal_augmented = "/Users/titouan/Desktop/HDA/Term_2/2-ML/Project/Pitch_Shift_train/"
files = os.listdir(wavs_signal_augmented)
files = [f for f in files if f.endswith('.wav')]

# Filepath for where to save melspectograms: output
augmentedData = "/Users/titouan/Desktop/HDA/Term_2/2-ML/Project/MelSpecs_train_augmented/"

# Filepath for where labels (0: no COVID, 1: COVID) for each melspectogram will be saved
labels_mels_signal_augmented = "/Users/titouan/Desktop/HDA/Term_2/2-ML/Project/MelSpecs_labels_train_augmented.csv"
SpectAugment(wavs_signal_augmented,
             files,
             20,
             augmentedData,
             labels_mels_signal_augmented,
             mean_signal_length+1)

Converting: |██████████████████████████████████████████████████| 100.0% 
